In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyentrp import entropy as ent
import antropy as ant
import statsmodels.api as sm
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load Patient 14 data from text file
eeg_data = np.loadtxt('D:\\users\\meyy2\\Documents\\University 2024 - Year 2\\Research Project\\Patient 14 EEG.txt')

from scipy.signal import butter, filtfilt

# Define band filter
def band_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Sampling frequency = 128 Hz
fs = 128

# Apply filters for each band
delta_band = band_filter(eeg_data, 0.5, 4, fs) # 0.5 to 4 Hz
theta_band = band_filter(eeg_data, 4, 8, fs) # 4 to 8 Hz
alpha_band = band_filter(eeg_data, 8, 13, fs) # 8 to 13 Hz
beta_band = band_filter(eeg_data, 13, 30, fs) # 13 to 30 Hz
gamma_band = band_filter(eeg_data, 30, 63, fs) # 30 to 63 Hz (within Nyquist range)

# Create overlapping 56 s windows of different bands, advancing by 1 s
def moving_window(data, window_size, step_size, fs):
    num_points = len(data)
    window_length = window_size * fs # Length of window in data points
    step_length = step_size * fs # Length of step in data points
    windows = []
    for start in range(0, num_points - window_length + 1, step_length):
        windows.append(data[start:start + window_length])
    return np.array(windows)

window_size = 56 # Length of window in seconds
step_size = 1 # Length of step in seconds
alpha_windows = moving_window(alpha_band, window_size, step_size, fs)
beta_windows = moving_window(beta_band, window_size, step_size, fs)
delta_windows = moving_window(delta_band, window_size, step_size, fs)
theta_windows = moving_window(theta_band, window_size, step_size, fs)
gamma_windows = moving_window(gamma_band, window_size, step_size, fs)

# Load BIS data of Patient 14
patient14_bis = np.loadtxt('D:\\users\\meyy2\\Documents\\University 2024 - Year 2\\Research Project\\Patient 14 BIS.txt')

# Align BIS and PE values by disregarding the first 55 BIS values, as each PE value will represent the end of the window
bis_aligned = patient14_bis[55:55+3001]

# Load Patient 12 data
eeg_data_patient12 = np.loadtxt('D:\\users\\meyy2\\Documents\\University 2024 - Year 2\\Research Project\\Patient 12 EEG data.txt')
bis_patient12 = np.loadtxt('D:\\users\\meyy2\\Documents\\University 2024 - Year 2\\Research Project\\Patient 12 BIS.txt')

# Apply filters for each band for Patient 12
delta_band_patient12 = band_filter(eeg_data_patient12, 0.5, 4, fs) # 0.5 to 4 Hz
theta_band_patient12 = band_filter(eeg_data_patient12, 4, 8, fs) # 4 to 8 Hz
alpha_band_patient12 = band_filter(eeg_data_patient12, 8, 13, fs) # 8 to 13 Hz
beta_band_patient12 = band_filter(eeg_data_patient12, 13, 30, fs) # 13 to 30 Hz
gamma_band_patient12 = band_filter(eeg_data_patient12, 30, 63, fs) # 30 to 63 Hz (within Nyquist range

# Patient 12 moving windows
alpha_windows_patient12 = moving_window(alpha_band_patient12, window_size, step_size, fs)
beta_windows_patient12 = moving_window(beta_band_patient12, window_size, step_size, fs)
delta_windows_patient12 = moving_window(delta_band_patient12, window_size, step_size, fs)
theta_windows_patient12 = moving_window(theta_band_patient12, window_size, step_size, fs)
gamma_windows_patient12 = moving_window(gamma_band_patient12, window_size, step_size, fs)

# Prepare data
bis_series = pd.Series(bis_aligned, name='BIS')
bis_aligned_patient12 = bis_patient12[55:55+len(beta_windows_patient12)]

In [4]:
!python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------- ----- 1.6/1.8 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 9.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [10]:
!pip install entropyhub

In [18]:
# Step 1: Create embedding vectors
def create_embedding(time_series, m):
    return np.array([time_series[i:i + m] for i in range(len(time_series) - m + 1)])

# Step 2: Calculate fuzzy similarity between two vectors
def fuzzy_similarity(x, y, r):
    return np.exp(-np.linalg.norm(x - y) ** 2 / (r ** 2))

# Step 3: Compute Fuzzy Entropy
def fuzzy_entropy(time_series, m, r):
    n = len(time_series)
    embedding = create_embedding(time_series, m)
    print("HIii")
    # Initialize fuzzy similarity matrix
    similarity_matrix = np.zeros((n - m, n - m))
    print("sim")
    # Step 4: Calculate similarity between all pairs of vectors
    for i in range(n - m):
        for j in range(i, n - m):
            similarity_matrix[i, j] = fuzzy_similarity(embedding[i], embedding[j], r)
            similarity_matrix[j, i] = similarity_matrix[i, j]
    print("ues")
    # Step 5: Calculate probability distribution
    prob_matrix = similarity_matrix / similarity_matrix.sum(axis=1, keepdims=True)
    
    # Step 6: Calculate Fuzzy Entropy
    entropy = -np.sum(prob_matrix * np.log(prob_matrix + 1e-10))  # Adding small epsilon to avoid log(0)
    print("hiiiiii")
    return entropy

# Set parameters
m = 2  # Embedding dimension
r = 0.2  # Threshold for fuzzy similarity (can be based on standard deviation)

fe_values_beta = []

print("Hi")

for i, window in enumerate(beta_windows):
    fe = fuzzy_entropy(window, 2, 0.2)
    fe_values_beta.append(fe)
    print(f'Processed window {i+1}/{len(beta_windows)}: value {fe}')

fe_values_beta = np.array(fe_values_beta)

Hi
HIii
sim


KeyboardInterrupt: 

In [ ]:
# Create the SHE index for beta band
she_df_beta = pd.DataFrame(she_values_beta, columns=['SHE']) # Convert SHE numpy array to pandas DataFrame
SHE_beta = she_df_beta.values.reshape(-1, 1)
SHE_model_beta = LinearRegression()
SHE_model_beta.fit(SHE_beta, bis_series)
SHE_index_beta = SHE_model_beta.predict(SHE_beta)

# Calculate the regression coefficients
k_SHE_beta = SHE_model_beta.coef_[0]
b_SHE_beta = SHE_model_beta.intercept_
print(f"SHE Index Equation: BIS = {k_SHE_beta} * SHE1 + {b_SHE_beta}")

# Plot BIS Index vs. SHE Index
plt.figure(figsize=(12, 6))
plt.plot(bis_series, label='BIS Index', color='blue')
plt.plot(SHE_index_beta, label='SHE Index', color='red', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Index Value')
plt.title('BIS Index vs. SHE Index Over Time for Patient 14 (Beta Band)')
plt.legend()
plt.show()

# Calculate Root Mean Square Error (RMSE) for the combined model
rmse_she_beta = np.sqrt(mean_squared_error(bis_series, SHE_index_beta))
print("SHE Beta Training Model RMSE: ", rmse_she_beta)

# Calculate Pearson correlation coefficient
